## 実験用の誤りを含む/含まない文ペアデータセットを作成する

データ読み込み

In [ ]:
import pandas as pd

df_typo_dataset_train = pd.read_json(
    "../../../../dataset/original/japanese_wikipedia_typo_dataset/train.jsonl",
    lines=True,
)
df_typo_dataset_test = pd.read_json(
    "../../../../dataset/original/japanese_wikipedia_typo_dataset/test.jsonl",
    lines=True,
)

中身を確認

In [ ]:
df_typo_dataset_train.head()

In [ ]:
df_typo_dataset_test.head()

縦に結合

In [ ]:
# データセット結合
df_typo_dataset = pd.concat(
    [df_typo_dataset_train, df_typo_dataset_test], ignore_index=True
)

タイポ部分までの文章を切り出す
- "diffs":[{'pre':'token1', 'post':'token2'}]  
の値をもとに、  
'pre_text'と'post_text'について、それぞれ'token1'と'token2'より後の文字を削除する  
(token1, token2 までは含める)

In [ ]:
def find_diff_position(text1: str, text2: str) -> int:
    """2つのテキストを比較して、最初に異なる文字の位置を返す

    Args:
        text1: 比較する1つ目のテキスト
        text2: 比較する2つ目のテキスト

    Returns:
        最初に異なる文字の位置（インデックス）

    """
    min_len = min(len(text1), len(text2))

    for i in range(min_len):
        if text1[i] != text2[i]:
            return i

    # 全て一致している場合は短い方の長さを返す
    return min_len


def extract_before_typo_token(
    pre_text: str, post_text: str, diffs: list[dict], key: str
) -> str:
    """pre_textとpost_textを比較し、diffs の値をもとに、タイポ部分までの文章を切り出す

    （タイポトークンを含む）

    Args:
        pre_text: タイポ前のテキスト
        post_text: タイポ後のテキスト
        diffs: タイポ情報のリスト [{'pre': 'token1', 'post': 'token2'}, ...]
        key: 'pre' または 'post' を指定

    Returns:
        タイポ部分までの文章（タイポトークンを含む）

    """
    if not diffs:
        return pre_text if key == "pre" else post_text

    # 使用するテキストを選択
    text = pre_text if key == "pre" else post_text

    # 最初のdiffのトークンを取得
    first_diff = diffs[0]
    token = first_diff.get(key, "")

    if not token:
        return text

    # pre_textとpost_textで異なる位置を見つける
    diff_pos = find_diff_position(pre_text, post_text)

    # 異なる位置からトークン長だけ取得して確認
    token_len = len(token)
    if diff_pos + token_len <= len(text):
        actual_token = text[diff_pos : diff_pos + token_len]
        if actual_token == token:
            # トークンを含めてその位置までを返す
            return text[: diff_pos + token_len]

    # フォールバック: 最初の出現位置を使用
    idx = text.find(token)
    if idx != -1:
        return text[: idx + len(token)]

    return text

In [ ]:
# pre_textとpost_textからタイポ部分までを抽出(diffsを使用、同じ位置で切り出し)
df_typo_dataset["pre_text_truncated"] = df_typo_dataset.apply(
    lambda row: extract_before_typo_token(
        row["pre_text"], row["post_text"], row["diffs"], "pre"
    ),
    axis=1,
)

df_typo_dataset["post_text_truncated"] = df_typo_dataset.apply(
    lambda row: extract_before_typo_token(
        row["pre_text"], row["post_text"], row["diffs"], "post"
    ),
    axis=1,
)

In [ ]:
df_typo_dataset.head()

In [ ]:
df_typo_dataset.shape

In [ ]:
# pre_text_truncated と post_text_truncated の文字数が同じもののみ抽出
df_typo_dataset = df_typo_dataset[
    df_typo_dataset["pre_text_truncated"].str.len()
    == df_typo_dataset["post_text_truncated"].str.len()
]

In [ ]:
df_typo_dataset.shape

In [ ]:
# 最小文字数の定義
MIN_TEXT_LENGTH = 20

# 最小文字数以上のデータに限定
df_typo_dataset = df_typo_dataset[df_typo_dataset["pre_text_truncated"].str.len() >= MIN_TEXT_LENGTH]

In [ ]:
df_typo_dataset.shape

保存

In [ ]:
# 使用するカラムだけ抽出
df_typo_dataset = df_typo_dataset[
    ["category", "pre_text_truncated", "post_text_truncated"]
]

# 名前の変更
df_typo_dataset = df_typo_dataset.rename(
    columns={"pre_text_truncated": "typo_text", "post_text_truncated": "no_typo_text"}
)

In [ ]:
df_typo_dataset["category"].value_counts()

In [ ]:
df_typo_dataset.to_json(
    "../../../../dataset/robustness_against_input_errors/typo_dataset.jsonl",
    orient="records",
    lines=True,
)